A potential drawback with one-hot encoded feature vector approaches such as N-Grams, bag of words and TF-IDF approach is that the feature vector for each document can be huge. For instance, if you have a half million unique words in your corpus and you want to represent a sentence that contains 10 words, your feature vector will be a half million dimensional one-hot encoded vector where only 10 indexes will have 1. This is a wastage of space and increases algorithm complexity exponentially resulting in the curse of dimentionality.

In word embeddings, every word is represented as an n-dimensional dense vector. The words that are similar will have similar vector. Word embeddings techniques such as GloVe and Word2Vec have proven to be extremely efficient for converting words into corresponding dense vectors. The vector size is small and none of the indexes in the vector is actually empty.

he Keras library contains a layer called Embedding(). The embedding layer is implemented in the form of a class in Keras and is normally used as a first layer in the sequential model for NLP tasks.

In [0]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [0]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]

In [0]:
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

the first parameter to the Embedding() layer is the vocabulary, or number of unique words in the corpus. Let's first find the total number of words in our corpus:

In [0]:
import nltk

nltk.download('punkt')
  
from nltk.tokenize import word_tokenize

all_words = []
for sent in corpus:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
unique_words = set(all_words)
print(len(unique_words))

45


In the output you will see "45", which is the number of unique words in our corpus. We will add a buffer of 5 to our vocabulary size and will set the value of vocab_length to 50.

One way to convert text to numbers is by using the one_hot function from the keras.preprocessing.text library. The function takes sentence and the total length of the vocabulary and returns the sentence in numeric form.

In [0]:
embedded_sentences = [one_hot(sent, 50) for sent in corpus]
print(embedded_sentences )

[[10, 34, 7, 19, 24], [19, 2, 29, 45, 33, 13, 8], [21, 43, 19, 8, 34, 21], [28, 43], [12, 20, 10, 46, 33, 13, 8], [23, 21, 45, 24], [32, 20, 8, 21, 49, 24], [8, 34, 21, 24, 24], [22, 32], [48, 4, 7], [6, 21], [8, 29, 21, 7], [33, 15, 29, 13, 19, 24], [19, 24, 29, 22], [33, 24, 29, 5], [19, 32, 34, 6]]


In [0]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

In [0]:
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

[[10 34  7 19 24  0  0]
 [19  2 29 45 33 13  8]
 [21 43 19  8 34 21  0]
 [28 43  0  0  0  0  0]
 [12 20 10 46 33 13  8]
 [23 21 45 24  0  0  0]
 [32 20  8 21 49 24  0]
 [ 8 34 21 24 24  0  0]
 [22 32  0  0  0  0  0]
 [48  4  7  0  0  0  0]
 [ 6 21  0  0  0  0  0]
 [ 8 29 21  7  0  0  0]
 [33 15 29 13 19 24  0]
 [19 24 29 22  0  0  0]
 [33 24 29  5  0  0  0]
 [19 32 34  6  0  0  0]]


In [0]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

In [0]:
model = Sequential()
model.add(Embedding(50, 20, input_length=length_long_sentence))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7, 20)             1000      
_________________________________________________________________
flatten_1 (Flatten)          (None, 140)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 141       
Total params: 1,141
Trainable params: 1,141
Non-trainable params: 0
_________________________________________________________________


we create a Sequential model and add the Embedding layer as the first layer to the model. The length of the vocabulary is specified by the vocab_length parameter. The dimension of each word vector will be 20 and the input_length will be the length of the longest sentence, which is 7. Next, the Embedding layer is flattened so that it can be directly used with the densely connected layer. Since it is a binary classification problem, we use the sigmoid function as the loss function at the dense layer.

In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7, 20)             1000      
_________________________________________________________________
flatten_1 (Flatten)          (None, 140)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 141       
Total params: 1,141
Trainable params: 1,141
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)




Epoch 1/100





16/16 [==============================] - 4s 265ms/step - loss: 0.6903 - acc: 0.5000
Epoch 2/100
16/16 [==============================] - 0s 508us/step - loss: 0.6865 - acc: 0.5625
Epoch 3/100
16/16 [==============================] - 0s 335us/step - loss: 0.6827 - acc: 0.6875
Epoch 4/100
16/16 [==============================] - 0s 361us/step - loss: 0.6789 - acc: 0.7500
Epoch 5/100
16/16 [==============================] - 0s 521us/step - loss: 0.6751 - acc: 0.8125
Epoch 6/100
16/16 [==============================] - 0s 382us/step - loss: 0.6714 - acc: 0.8125
Epoch 7/100
16/16 [==============================] - 0s 275us/step - loss: 0.6676 - acc: 0.8125
Epoch 8/100
16/16 [==============================] - 0s 348us/step - loss: 0.6638 - acc: 0.8125
Epoch 9/100
16/16 [==============================] - 0s 304us/step - loss: 0.6601 - acc: 0.8750
Epoch 10/100
16/16 [==============================] - 0s 325us/step - loss: 0.6563 - acc: 0.8750
Epoch 11/100
16/16 [===========

In [0]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000
